<a href="https://colab.research.google.com/github/H-Levi/NLP_Project1/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import json
import wikipedia
import wikipediaapi
from nltk.classify import NaiveBayesClassifier
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# Assign a user_agent
user_agent = 'hailegabriel/1.0 (bentitan98@gmail.com)'
wiki_wiki = wikipediaapi.Wikipedia(user_agent, 'en', wikipediaapi.ExtractFormat.WIKI)


my_file = {}
# Search for titles in wikipedia
for items in wikipedia.search('Medical', results=360):
    # Extract the summary of the content of the selected titles
    my_file[items] = wiki_wiki.page(items).summary
# create a JSON file and write the content summary i
with open('medical.json', 'w') as medical:
    json.dump(my_file, medical, indent=4)

user_agent = 'hailegabriel/1.0 (bentitan98@gmail.com)'
wiki_wiki = wikipediaapi.Wikipedia(user_agent, 'en', wikipediaapi.ExtractFormat.WIKI)

my_file = {}
non_medical_titles = ['Sport', 'Technology', 'Art', 'Entertainment', 'Finance', 'Travel', 'Fashion', 'Business', 'Education']

for content in non_medical_titles:
    for items in wikipedia.search(content, results=40):
        key = f"{content}_{items}"  # Use a unique key based on content type and title
        my_file[key] = wiki_wiki.page(items).summary

with open('non_medical.json', 'w') as non_medical:
    json.dump(my_file, non_medical, indent=4)


In [5]:
!pip install wikipedia-api
gi

In [15]:

# Load medical data from JSON
with open('medical.json', 'r') as file1:
    medical_data_dict = json.load(file1)
    medical_data = list(medical_data_dict.values())

# Load non-medical data from JSON
with open('non_medical.json', 'r') as file2:
    non_medical_data_dict = json.load(file2)
    non_medical_data = list(non_medical_data_dict.values())


# Preprocess data
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalpha()]  # Remove non-alphabetic characters
    words = [word for word in words if word not in stop_words]  # Remove stop words
    words = [stemmer.stem(word) for word in words]  # Stemming
    words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatization
    return words

# Extract features from text
def extract_features(text):
    features = {}
    for word in preprocess_text(text):
        features[word] = True
    return features

# Create labeled dataset to label as well as combine both data sets into one
labeled_data = [(text, 'medical') for text in medical_data] + [(text, 'nonmedical') for text in non_medical_data]

# Split dataset into training and testing sets
train_set, test_set = train_test_split(labeled_data, test_size=0.2, random_state=42)

# Train Naive Bayes classifier
nb_classifier = NaiveBayesClassifier.train([(extract_features(text), label) for (text, label) in train_set])

# Evaluate Naive Bayes classifier
nb_predictions = [nb_classifier.classify(extract_features(text)) for (text, label) in test_set]
nb_accuracy = accuracy_score([label for (text, label) in test_set], nb_predictions)
nb_report = classification_report([label for (text, label) in test_set], nb_predictions)

print("Naive Bayes Accuracy:", nb_accuracy)
print("Naive Bayes Report:\n", nb_report)

# Additional testing sets for manual test
additional_sentences = [
    "I love to play football.",
    "The human body has 206 bones.",
    "The capital of France is Paris.",
    "Artificial intelligence is reshaping industries.",
    "The stock market experienced a sharp decline yesterday.",
]

# Test the models with additional sentences
for sentence in additional_sentences:
    # Extract features from the sentence
    features = extract_features(sentence)

    # Predict with Naive Bayes
    nb_prediction = nb_classifier.classify(features)
    print(f"Naive Bayes Prediction for '{sentence}': {nb_prediction}")





Naive Bayes Accuracy: 0.9305555555555556
Naive Bayes Report:
               precision    recall  f1-score   support

     medical       0.97      0.90      0.94        82
  nonmedical       0.88      0.97      0.92        62

    accuracy                           0.93       144
   macro avg       0.93      0.94      0.93       144
weighted avg       0.93      0.93      0.93       144

Naive Bayes Prediction for 'I love to play football.': nonmedical
Naive Bayes Prediction for 'The human body has 206 bones.': medical
Naive Bayes Prediction for 'The capital of France is Paris.': nonmedical
Naive Bayes Prediction for 'Artificial intelligence is reshaping industries.': nonmedical
Naive Bayes Prediction for 'The stock market experienced a sharp decline yesterday.': nonmedical


In [13]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...


True